In [ ]:
import netCDF4 as nc
import numpy as np
from scipy.interpolate import interp1d


fnumbers = [f'{num:05}' for num in range(1461, 3261+1, 30)] # last valid file 10611

fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_'+str(fnumbers[0])+'.nc'
dataset = nc.Dataset(fname)
angle = dataset.variables['angle']
angle = angle[0,0]
lon_rho  = dataset.variables['lon_rho']
lon_rho = np.transpose(lon_rho, axes=(1, 0))
lat_rho  = dataset.variables['lat_rho']
lat_rho = np.transpose(lat_rho, axes=(1, 0))
z_r = np.load('/home/z5297792/MRes/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))
depths = z_r[150,150,:]

def depth_interpolator(M,z_r,depths):

    # Dimensions of M and z_r
    dim_i, dim_j, dim_sigma, dim_time = M.shape
    assert z_r.shape == (dim_i, dim_j, dim_sigma)

    # Initialize interpolated matrix
    M_interpolated = np.zeros((dim_i, dim_j, len(depths), dim_time))

    # Interpolate for each day (time dimension)
    for t in range(dim_time):
        for i in range(dim_i):
            for j in range(dim_j):
                # Get depth levels from z_r
                depth_levels = z_r[i, j, :]
                
                # Get values of M at each depth level for current time t
                values = M[i, j, :, t]
                
                # Perform interpolation
                if np.all(np.isnan(values)):
                    # Handle case where all values are NaN (e.g., outside valid data range)
                    interpolated_values = np.full(len(depths), np.nan)
                else:
                    # Use linear interpolation
                    interpolator = interp1d(depth_levels, values, kind='linear', bounds_error=False, fill_value=np.nan)
                    interpolated_values = interpolator(depths)
                
                # Assign interpolated values to M_interpolated for current time t
                M_interpolated[i, j, :, t] = interpolated_values
        M_interpolated[:,:, 0, t] = M[:,:,0,t]

    return M_interpolated

for fnumber in fnumbers:
    fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_'+str(fnumber)+'.nc'
    dataset = nc.Dataset(fname)
    u_east = dataset.variables['u_eastward']
    u_east = np.transpose(u_east, axes=(3, 2, 1, 0))
    u_east = np.flip(u_east, axis=2)
    v_north = dataset.variables['v_northward']
    v_north = np.transpose(v_north, axes=(3, 2, 1, 0))
    v_north = np.flip(v_north, axis=2)

    u_east = depth_interpolator(u_east, z_r, depths)
    v_north = depth_interpolator(v_north, z_r, depths)

    # ROTATE U_EAST AND V_NORTH TO x,y AXIS
    u = np.full_like(u_east, np.nan)
    v = np.full_like(v_north, np.nan)

    WIDTH, LENGTH, SIGMAS, TOTAL_NUM_DAYS = u.shape

    for TIME in range(30):
        u[:, :, :, TIME] = v_north[:, :, :, TIME] * np.sin(angle) + u_east[:, :, :, TIME] * np.cos(angle)
        v[:, :, :, TIME] = v_north[:, :, :, TIME] * np.cos(angle) - u_east[:, :, :, TIME] * np.sin(angle)

    del u_east, v_north

    np.save('/srv/scratch/z5297792/Climatology/u_v/u_'+str(fnumber)+'.npy', u)
    np.save('/srv/scratch/z5297792/Climatology/u_v/v_'+str(fnumber)+'.npy', v)
    print(fnumber)


01461
01491
01521
01551
01581
01611
01641
01671
01701
01731
01761
01791
01821
01851
01881
01911
